In [ ]:
# Clone Repo
!git clone https://github.com/Sharp-4rth/temporal_belief_analysis.git

In [ ]:
# Need to restart after:
!pip install convokit[llm]
!pip install convokit
!pip install statsmodels

In [ ]:
import sys
import os
os.chdir('/content/temporal_belief_analysis/notebooks')
print("Changed working directory to:", os.getcwd())
# Absolute path to src directory
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

In [ ]:
import time
!pip install gdown
import zipfile
import nltk
from nltk.corpus import stopwords
from convokit import Corpus, download
import convokit
from temporal_belief.core.timeline_building import TimelineBuilder
from temporal_belief.core.change_detection import ChangeDetector
from temporal_belief.core.window_extraction import WindowExtractor
from temporal_belief.core.op_path_pairing import OpPathPairer
from temporal_belief.data.preprocessors import ChangeDetectorPreprocessor
from temporal_belief.data.preprocessors import PairPreprocessor
from temporal_belief.data.preprocessors import ExtractFeatures
from temporal_belief.data.preprocessors import GroupPreprocessor
from temporal_belief.core.interplay import Interplay
import numpy as np
nltk.download('stopwords')

In [ ]:
# Download 'r/PoliticalDiscussion' corpus with stance labels to root, unzip with python:
!gdown "https://drive.google.com/file/d/1AIrstrzE259fcVyxJQW4-RwvAkoUyK1x/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances_fine_tuned.zip" --fuzzy
zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_stances_fine_tuned.zip").extractall("/content/temporal_belief_analysis")

In [ ]:
CORPUS_PATH = "/content/temporal_belief_analysis/pd_corpus_with_stances_fine_tuned"
corpus = Corpus(filename=CORPUS_PATH)

In [13]:
# Build per-user topic timelines:
timeline_builder = TimelineBuilder(corpus)
timelines = timeline_builder.build_timelines()

In [23]:
# A user to test with:
user_id = "HardCoreModerate"
topic = "Process & Actors (Meta)"
topic_timeline = timelines[user_id][topic]
topic_timeline = list(topic_timeline.items())

In [24]:
change_detector = ChangeDetector()
change_point = change_detector.detect_cusum_changes(topic_timeline)['change_points']
print(change_point)

['c4w912m']


In [33]:
window_extractor = WindowExtractor(corpus, timelines)
# window_extractor.build_global_user_conversations_index()
candidate_convos = window_extractor.get_conversations_around_change_point(change_point=change_point, corpus=corpus)
# print(f"Candidate convos: {[convo for convo in candidate_convos]}")

TypeError: unhashable type: 'list'

In [27]:
op_path_pairer = OpPathPairer(corpus, timelines)
pair_preprocessor = PairPreprocessor()

In [ ]:
groups_preprocessor = GroupPreprocessor()
groups = change_detector.get_two_groups(timelines)
groups_tuple = (groups['with_changes'], groups['no_changes'])
groups_tuple = groups_preprocessor.filter_groups(groups, groups_tuple)
target_utterances = groups_preprocessor.get_target(groups_tuple)

In [ ]:
feature_extractor = ExtractFeatures()
persuasion_analyzer = Interplay()

In [ ]:
from tqdm import tqdm
stop_words_set = set(stopwords.words('english'))

group_means = []
group_scores = []

# For each group
for group_idx, group in enumerate(tqdm(groups_tuple, desc="Processing groups")):
    # Initialize dictionary for this group's scores (one score per utterance)
    current_group_scores = {
        'interplay': [],
        'politeness': [],
        'argument_complexity': [],
        'evidence_markers': [],
        'hedging': []
    }

    utterances_processed = 0
    target_reached = False  # Flag to control all nested loops

    for user_id, topic_timelines in group.items():
        if target_reached:  # Check flag at user level
            break

        user_start_time = time.time()
        user_change_points = 0

        for topic_timeline in topic_timelines.values():
            if target_reached:  # Check flag at topic level
                break

            for change_point in topic_timeline.keys():  # Each utterance/change point
                if utterances_processed >= target_utterances:
                    target_reached = True  # Set flag instead of just breaking
                    break

                # utts_num += 1
                user_change_points += 1
                utterances_processed += 1

                # Window extraction
                start_time = time.time()
                try:
                    candidate_convos = window_extractor.get_conversations_around_change_point(
                        change_point=change_point, corpus=corpus, test=True
                    )
                    window_time = time.time() - start_time
                    print(f'Window extraction: {window_time:.3f}s')
                except ValueError as e:
                    print(f"Skipping change point {change_point}: {e}")
                    continue

                # Path extraction
                start_time = time.time()
                timeout_duration = 0.25
                op_path_pairs = []

                for candidate_convo in candidate_convos:
                    if time.time() - start_time > timeout_duration:
                        print(f"Path extraction timeout reached ({timeout_duration}s)")
                        break

                    try:
                        op_path_pairs.extend(op_path_pairer.extract_rooted_path_from_candidate_convos(
                            [candidate_convo], user_id
                        ))
                    except ValueError as e:
                        print(f"Skipping conversation {candidate_convo.id}: {e}")
                        continue

                path_time = time.time() - start_time
                print(f'Path extraction: {path_time:.3f}s')

                # Preprocessing
                start_time = time.time()
                preprocessed_pairs = pair_preprocessor.concatenate_path_in_all_pairs(op_path_pairs)
                preprocess_time = time.time() - start_time
                print(f'Preprocessing: {preprocess_time:.3f}s')

                # Feature extraction - collect ALL scores for this utterance
                start_time = time.time()
                utterance_interplay_scores = []
                utterance_politeness_scores = []
                utterance_complexity_scores = []
                utterance_evidence_scores = []
                utterance_hedging_scores = []

                for op, paths in preprocessed_pairs:
                    for k, concatenated_utts in paths.items():
                        # Extract features
                        interplay_features = persuasion_analyzer.calculate_interplay_features(
                            op.text, concatenated_utts, stop_words_set
                        )
                        politeness_features = feature_extractor.get_politeness_features(concatenated_utts)
                        complexity_features = feature_extractor.extract_argument_complexity_features(concatenated_utts)
                        evidence_features = feature_extractor.extract_evidence_features(concatenated_utts)
                        hedging_features = feature_extractor.extract_hedging_features(concatenated_utts)

                        # Calculate scores
                        interplay_score = persuasion_analyzer.calculate_persuasion_score(interplay_features)
                        politeness_score = sum(politeness_features.values())
                        complexity_score = feature_extractor.calculate_complexity_score(complexity_features)
                        evidence_score = feature_extractor.calculate_evidence_score(evidence_features)
                        hedging_score = feature_extractor.calculate_hedging_score_from_features(hedging_features)

                        # Collect all scores for this utterance
                        utterance_interplay_scores.append(interplay_score)
                        utterance_politeness_scores.append(politeness_score)
                        utterance_complexity_scores.append(complexity_score)
                        utterance_evidence_scores.append(evidence_score)
                        utterance_hedging_scores.append(hedging_score)

                feature_time = time.time() - start_time

                # Take mean across all paths for this single utterance
                start_time = time.time()
                if utterance_interplay_scores:  # Only if we have scores
                    # One score per utterance (mean of all conversation paths)
                    utterance_mean_interplay = np.mean(utterance_interplay_scores)
                    utterance_mean_politeness = np.mean(utterance_politeness_scores)
                    utterance_mean_complexity = np.mean(utterance_complexity_scores)
                    utterance_mean_evidence = np.mean(utterance_evidence_scores)
                    utterance_mean_hedging = np.mean(utterance_hedging_scores)

                    # Add ONE score per utterance to group scores
                    current_group_scores['interplay'].append(utterance_mean_interplay)
                    current_group_scores['politeness'].append(utterance_mean_politeness)
                    current_group_scores['argument_complexity'].append(utterance_mean_complexity)
                    current_group_scores['evidence_markers'].append(utterance_mean_evidence)
                    current_group_scores['hedging'].append(utterance_mean_hedging)

                    print(f"Utterance {change_point}: {len(utterance_interplay_scores)} paths -> 1 mean score")
                    print(f"Group {group_idx + 1}: {utterances_processed}/{target_utterances} utterances processed")
                else:
                    print(f"Utterance {change_point}: No valid paths found, skipping")

                scoring_time = time.time() - start_time
                print(f'Scoring: {scoring_time:.3f}s')

                # Print total time for this change point
                total_time = window_time + path_time + preprocess_time + feature_time + scoring_time
                print(f'TOTAL for utterance: {total_time:.3f}s\n')

        user_total_time = time.time() - user_start_time
        if user_change_points > 0:  # Only print if user had utterances
            print(f'USER {user_id}: {user_total_time:.3f}s ({user_change_points} utterances)')

    # Calculate means for each predictor for this group
    group_mean = {}
    for predictor_name, scores in current_group_scores.items():
        if scores:
            group_mean[predictor_name] = np.mean(scores)
        else:
            group_mean[predictor_name] = 0

    print(f"\nGroup {group_idx + 1} final sample sizes:")
    for predictor_name, scores in current_group_scores.items():
        print(f"  {predictor_name}: n={len(scores)}")

    print(f"Group {group_idx + 1}: Processed exactly {utterances_processed} utterances")

    group_means.append(group_mean)
    group_scores.append(current_group_scores)

In [ ]:
groups_preprocessor.run_statistical_comparison(group_scores)